In [57]:
import gzip
import sys
import os
import json
import math
import calendar
import codecs
import datetime
import scipy.stats
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
from scipy.stats import linregress
import pandas as pd
import glob
%matplotlib inline

# PHEME Message Features

In [54]:
annotations = pd.read_json("../../Data/pheme-rumour-scheme-dataset/annotations/en-scheme-annotations.json", lines=True)

In [120]:
annotations.head(5)

,certainty,event,evidentiality,responsetype-vs-previous,responsetype-vs-source,support,threadid,tweetid
0,somewhat-certain,putinmissing,url-given,NaN,NaN,supporting,577258317942149120,577258317942149120
1,somewhat-certain,putinmissing,url-given,NaN,NaN,supporting,576755174531862528,576755174531862528
2,somewhat-certain,putinmissing,url-given,NaN,NaN,supporting,576319832800555008,576319832800555008
3,certain,putinmissing,url-given,NaN,NaN,denying,576513463738109952,576513463738109952
4,certain,charliehebdo,url-given,NaN,NaN,supporting,552783667052167168,552783667052167168


In [122]:
annotations.rename({"tweetid": "id"}, axis='columns')

,certainty,event,evidentiality,responsetype-vs-previous,responsetype-vs-source,support,threadid,id
0,somewhat-certain,putinmissing,url-given,NaN,NaN,supporting,577258317942149120,577258317942149120
1,somewhat-certain,putinmissing,url-given,NaN,NaN,supporting,576755174531862528,576755174531862528
2,somewhat-certain,putinmissing,url-given,NaN,NaN,supporting,576319832800555008,576319832800555008
3,certain,putinmissing,url-given,NaN,NaN,denying,576513463738109952,576513463738109952
4,certain,charliehebdo,url-given,NaN,NaN,supporting,552783667052167168,552783667052167168
5,certain,charliehebdo,url-given,NaN,NaN,supporting,552793679082311680,552793679082311680
6,somewhat-certain,charliehebdo,source-quoted,NaN,NaN,supporting,553548567420628992,553548567420628992
7,certain,charliehebdo,url-given,NaN,NaN,supporting,552832817089236992,552832817089236992
8,certain,charliehebdo,url-given,NaN,NaN,supporting,552833028201144320,552833028201144320
9,somewhat-certain,charliehebdo,witnessed,NaN,NaN,denying,553184482241814528,553184482241814528


In [85]:
frames = []
for subdir, dirs, files in os.walk("../../Data/pheme-rumour-scheme-dataset/threads/en"):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".json") and filepath != subdir + os.sep + "annotation.json":
            if filepath != subdir + os.sep + "structure.json":
                if filepath != subdir + os.sep + "retweets.json":
                    print("Currently reading: " + filepath)
                    frames.append(pd.read_json(filepath, lines=True))

tweets = pd.concat(frames)

Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552783667052167168\reactions\552785374507175936.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552783667052167168\reactions\552786226546495488.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552783667052167168\reactions\552789153403109376.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552783667052167168\reactions\552789345628069888.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552783667052167168\reactions\552790281276628992.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552783667052167168\reactions\552792862610694144.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552783667052167168\reactions\552943855021330432.json
Currently reading: ../../Data/pheme-rumour-scheme-datas

Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552792544132997121\reactions\552805439474515968.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552792544132997121\reactions\552805557896482816.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552792544132997121\reactions\552805909425307648.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552792544132997121\reactions\552806009157464064.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552792544132997121\reactions\552806259767123968.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552792544132997121\reactions\552806438629048320.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552792544132997121\reactions\552806489363337217.json
Currently reading: ../../Data/pheme-rumour-scheme-datas

Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552807385496973312.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552807494545076225.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552807670710038528.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552807834078171136.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552808195710664704.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552808282440859648.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552808456894169089.json
Currently reading: ../../Data/pheme-rumour-scheme-datas

Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552819494372921346.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552819783675432960.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552819812435369987.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552819928315617281.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552819966006001664.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552820020904861696.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552805488631758849\reactions\552820213129809921.json
Currently reading: ../../Data/pheme-rumour-scheme-datas

Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552806309540528128\reactions\552812407064899585.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552806309540528128\reactions\552812425758912512.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552806309540528128\reactions\552812983370268672.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552806309540528128\reactions\552813608694857730.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552806309540528128\reactions\552814267020636160.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552806309540528128\reactions\552814555764506624.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\charliehebdo\552806309540528128\reactions\552814637696450560.json
Currently reading: ../../Data/pheme-rumour-scheme-datas

...

Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544462330105712640\reactions\544599722577514496.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544462330105712640\reactions\544757182521032704.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544462330105712640\reactions\544759066950524928.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544462330105712640\reactions\544761239641923584.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544462330105712640\reactions\544761639031939072.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544462330105712640\reactions\544762521974890496.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544462330105712640\reactions\544762753890525184.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/thre

Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544504183341064192\reactions\544723320164204544.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544504183341064192\reactions\544795666946342913.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544504183341064192\reactions\544795839994925057.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544504183341064192\reactions\544798752855183360.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544504183341064192\reactions\544799211401641984.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544504183341064192\reactions\544805351518769152.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544504183341064192\reactions\544805630607753216.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/thre

Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544512910538838016\reactions\544513210490318848.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544512910538838016\reactions\544513540481355777.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544512910538838016\reactions\544514138186448896.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544512910538838016\reactions\544514139784478720.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544512910538838016\reactions\544514154485534721.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544512910538838016\reactions\544514368105611264.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544512910538838016\reactions\544514838509006849.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/thre

Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544517264054423552\reactions\544517553427865601.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544517264054423552\reactions\544517560364830720.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544517264054423552\reactions\544517577964531712.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544517264054423552\reactions\544517659174240256.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544517264054423552\reactions\544517684797267969.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544517264054423552\reactions\544517691906605056.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/threads/en\sydneysiege\544517264054423552\reactions\544517721862725632.json
Currently reading: ../../Data/pheme-rumour-scheme-dataset/thre

C:\Users\eric4\Anaconda3\envs\SmResearch\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [119]:
tweets.head(5)

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,filter_level,geo,id,...,metadata,place,possibly_sensitive,possibly_sensitive_appealable,retweet_count,retweeted,source,text,truncated,user
0,NaN,NaN,2015-01-07 11:14:38,"{'symbols': [], 'user_mentions': [{'id': 25067...",NaN,2,False,NaN,NaN,552785374507175936,...,NaN,NaN,NaN,NaN,1,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",MT @euronews France: 10 dead after shooting at...,False,"{'follow_request_sent': False, 'profile_use_ba..."
0,NaN,NaN,2015-01-07 11:18:01,"{'symbols': [], 'user_mentions': [{'id': 76987...",NaN,0,False,NaN,NaN,552786226546495488,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://itunes.apple.com/us/app/twitte...",@j0nathandavis They who? Stupid and partial op...,False,"{'follow_request_sent': False, 'profile_use_ba..."
0,NaN,NaN,2015-01-07 11:29:39,"{'symbols': [], 'user_mentions': [{'id': 25067...",NaN,0,False,NaN,NaN,552789153403109376,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com/download/android"" ...",@euronews @Massimilian2611 ci vuole una nuova ...,False,"{'follow_request_sent': False, 'profile_use_ba..."
0,NaN,NaN,2015-01-07 11:30:24,"{'symbols': [], 'user_mentions': [{'id': 33668...",NaN,0,False,NaN,NaN,552789345628069888,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","@nanoSpawn Socialists, Antisemites, anti zioni...",False,"{'follow_request_sent': False, 'profile_use_ba..."
0,NaN,NaN,2015-01-07 11:34:08,"{'symbols': [], 'user_mentions': [{'id': 40987...",NaN,0,False,NaN,NaN,552790281276628992,...,NaN,NaN,NaN,NaN,0,False,"<a href=""http://www.tweetcaster.com"" rel=""nofo...",@Nuno_Rodrigo5 @euronews islamistas o taraos?,False,"{'follow_request_sent': False, 'profile_use_ba..."


In [118]:
len(np.array(tweets.id.unique()))

5056

In [104]:
uniqueID = np.array(tweets.id)
annoID = np.array(annotations.tweetid)

In [110]:
found = False
for i in range(len(uniqueID)):
    for j in range(len(annoID)):
        if uniqueID[i] == annoID[j]:
            print("ID: {} is the same as ID: {}".format(uniqueID[i], annoID[j]))
            found = True
            break
        elif found == False and i == len(uniqueID)-1:
            print("{} does not exist in the Annotation array.".format(uniqueID[i]))

ID: 552785374507175936 is the same as ID: 552785374507175936
ID: 552786226546495488 is the same as ID: 552786226546495488
ID: 552789345628069888 is the same as ID: 552789345628069888
ID: 552792862610694144 is the same as ID: 552792862610694144
ID: 552943855021330432 is the same as ID: 552943855021330432
ID: 552783667052167168 is the same as ID: 552783667052167168
ID: 552785877748158464 is the same as ID: 552785877748158464
ID: 552788073340489728 is the same as ID: 552788073340489728
ID: 552791516360765440 is the same as ID: 552791516360765440
ID: 552792023078825984 is the same as ID: 552792023078825984
ID: 552792048173334528 is the same as ID: 552792048173334528
ID: 552792055122886656 is the same as ID: 552792055122886656
ID: 552792212056993792 is the same as ID: 552792212056993792
ID: 552792555944153088 is the same as ID: 552792555944153088
ID: 552792564554678272 is the same as ID: 552792564554678272
ID: 552792955052761088 is the same as ID: 552792955052761088
ID: 552793325733183488 i

ID: 552816420074291200 is the same as ID: 552816420074291200
ID: 552816523593916416 is the same as ID: 552816523593916416
ID: 552814494381256704 is the same as ID: 552814494381256704
ID: 552817849845436416 is the same as ID: 552817849845436416
ID: 552817939196678144 is the same as ID: 552817939196678144
ID: 552830099046035456 is the same as ID: 552830099046035456
ID: 552816020403269632 is the same as ID: 552816020403269632
ID: 552822567057510400 is the same as ID: 552822567057510400
ID: 552822679712313344 is the same as ID: 552822679712313344
ID: 552823713876303872 is the same as ID: 552823713876303872
ID: 552824008379736064 is the same as ID: 552824008379736064
ID: 552824073244246016 is the same as ID: 552824073244246016
ID: 552826863698317312 is the same as ID: 552826863698317312
ID: 552829975800578048 is the same as ID: 552829975800578048
ID: 552834187426689024 is the same as ID: 552834187426689024
ID: 552839935280300032 is the same as ID: 552839935280300032
ID: 552840084962443264 i

ID: 553467937575952384 is the same as ID: 553467937575952384
ID: 553469592488931328 is the same as ID: 553469592488931328
ID: 553496340580556800 is the same as ID: 553496340580556800
ID: 553503688946225152 is the same as ID: 553503688946225152
ID: 553504117230800896 is the same as ID: 553504117230800896
ID: 553514398871216128 is the same as ID: 553514398871216128
ID: 553535342049902592 is the same as ID: 553535342049902592
ID: 554412391279374336 is the same as ID: 554412391279374336
ID: 553461741917863936 is the same as ID: 553461741917863936
ID: 553481000379248640 is the same as ID: 553481000379248640
ID: 553467311261503488 is the same as ID: 553467311261503488
ID: 553471272353402880 is the same as ID: 553471272353402880
ID: 553471490734039040 is the same as ID: 553471490734039040
ID: 553473556345290752 is the same as ID: 553473556345290752
ID: 553474160530587648 is the same as ID: 553474160530587648
ID: 553475360936165376 is the same as ID: 553475360936165376
ID: 553478423411441664 i

ID: 553551642369753088 is the same as ID: 553551642369753088
ID: 553538279073935360 is the same as ID: 553538279073935360
ID: 553538493738405888 is the same as ID: 553538493738405888
ID: 553538596938846208 is the same as ID: 553538596938846208
ID: 553538823989493760 is the same as ID: 553538823989493760
ID: 553538987554385920 is the same as ID: 553538987554385920
ID: 553539141443788800 is the same as ID: 553539141443788800
ID: 553539293076279296 is the same as ID: 553539293076279296
ID: 553539507786903552 is the same as ID: 553539507786903552
ID: 553539728067551232 is the same as ID: 553539728067551232
ID: 553539730449924096 is the same as ID: 553539730449924096
ID: 553539762624012288 is the same as ID: 553539762624012288
ID: 553539889237884928 is the same as ID: 553539889237884928
ID: 553540050676219904 is the same as ID: 553540050676219904
ID: 553541495588802560 is the same as ID: 553541495588802560
ID: 553541672471396352 is the same as ID: 553541672471396352
ID: 553538058440941568 i

ID: 553592819622092800 is the same as ID: 553592819622092800
ID: 553594432743018496 is the same as ID: 553594432743018496
ID: 553595330009518080 is the same as ID: 553595330009518080
ID: 553595514739253248 is the same as ID: 553595514739253248
ID: 553596198956056576 is the same as ID: 553596198956056576
ID: 553596969978179584 is the same as ID: 553596969978179584
ID: 553597910504714240 is the same as ID: 553597910504714240
ID: 553588358044483584 is the same as ID: 553588358044483584
ID: 553588472330452992 is the same as ID: 553588472330452992
ID: 553588570368532480 is the same as ID: 553588570368532480
ID: 553588869409832960 is the same as ID: 553588869409832960
ID: 553589038196981760 is the same as ID: 553589038196981760
ID: 553589124125323264 is the same as ID: 553589124125323264
ID: 553589955029512192 is the same as ID: 553589955029512192
ID: 553591897458237440 is the same as ID: 553591897458237440
ID: 553593015416008704 is the same as ID: 553593015416008704
ID: 553588178687655936 i

...

ID: 525081032824209408 is the same as ID: 525081032824209408
ID: 525057081515061248 is the same as ID: 525057081515061248
ID: 525057109910495232 is the same as ID: 525057109910495232
ID: 525057571321688064 is the same as ID: 525057571321688064
ID: 525057624916520960 is the same as ID: 525057624916520960
ID: 525058373591367680 is the same as ID: 525058373591367680
ID: 525058435361292288 is the same as ID: 525058435361292288
ID: 525059328685133824 is the same as ID: 525059328685133824
ID: 525060851661668352 is the same as ID: 525060851661668352
ID: 525061525409525760 is the same as ID: 525061525409525760
ID: 525062334302412800 is the same as ID: 525062334302412800
ID: 525066880219115520 is the same as ID: 525066880219115520
ID: 525070441405095936 is the same as ID: 525070441405095936
ID: 525080429410664448 is the same as ID: 525080429410664448
ID: 525080746080620544 is the same as ID: 525080746080620544
ID: 525278489721966592 is the same as ID: 525278489721966592
ID: 525065014281908224 i

ID: 544277158747574272 is the same as ID: 544277158747574272
ID: 544277779517165568 is the same as ID: 544277779517165568
ID: 544280276914827264 is the same as ID: 544280276914827264
ID: 544269749405097984 is the same as ID: 544269749405097984
ID: 544271964890611712 is the same as ID: 544271964890611712
ID: 544272007013994496 is the same as ID: 544272007013994496
ID: 544272511912136704 is the same as ID: 544272511912136704
ID: 544272839889526784 is the same as ID: 544272839889526784
ID: 544272856528715776 is the same as ID: 544272856528715776
ID: 544274483289157632 is the same as ID: 544274483289157632
ID: 544280324360781824 is the same as ID: 544280324360781824
ID: 544292547955146752 is the same as ID: 544292547955146752
ID: 544300575592939520 is the same as ID: 544300575592939520
ID: 544304399456432128 is the same as ID: 544304399456432128
ID: 544271069146656768 is the same as ID: 544271069146656768
ID: 544271682358493184 is the same as ID: 544271682358493184
ID: 544273029765685248 i

ID: 544308033246220288 is the same as ID: 544308033246220288
ID: 544301453717041152 is the same as ID: 544301453717041152
ID: 544306128805724160 is the same as ID: 544306128805724160
ID: 544306274146713600 is the same as ID: 544306274146713600
ID: 544306589252198400 is the same as ID: 544306589252198400
ID: 544307524770738176 is the same as ID: 544307524770738176
ID: 544310038613852160 is the same as ID: 544310038613852160
ID: 544317774495875072 is the same as ID: 544317774495875072
ID: 544327005471526912 is the same as ID: 544327005471526912
ID: 544333035370721280 is the same as ID: 544333035370721280
ID: 544309751887429632 is the same as ID: 544309751887429632
ID: 544311947936538624 is the same as ID: 544311947936538624
ID: 544317391405330432 is the same as ID: 544317391405330432
ID: 544305745416581120 is the same as ID: 544305745416581120
ID: 544320468892020736 is the same as ID: 544320468892020736
ID: 544482722295844864 is the same as ID: 544482722295844864
ID: 544306402731507712 i

ID: 544350712365207552 is the same as ID: 544350712365207552
ID: 544352859773755392 is the same as ID: 544352859773755392
ID: 544354603211710464 is the same as ID: 544354603211710464
ID: 544355621936439296 is the same as ID: 544355621936439296
ID: 544358798895636480 is the same as ID: 544358798895636480
ID: 544358922904420352 is the same as ID: 544358922904420352
ID: 544367318193930240 is the same as ID: 544367318193930240
ID: 544367820407332864 is the same as ID: 544367820407332864
ID: 544358641772806144 is the same as ID: 544358641772806144
ID: 544358715659665408 is the same as ID: 544358715659665408
ID: 544362985162956800 is the same as ID: 544362985162956800
ID: 544363266244214784 is the same as ID: 544363266244214784
ID: 544417916796432384 is the same as ID: 544417916796432384
ID: 544419950816161792 is the same as ID: 544419950816161792
ID: 544452175754448896 is the same as ID: 544452175754448896
ID: 544452193089503232 is the same as ID: 544452193089503232
ID: 544452228632047616 i

ID: 544491585384505344 is the same as ID: 544491585384505344
ID: 544491587271544832 is the same as ID: 544491587271544832
ID: 544492317546393600 is the same as ID: 544492317546393600
ID: 544493109963661312 is the same as ID: 544493109963661312
ID: 544493474612256768 is the same as ID: 544493474612256768
ID: 544506524794908672 is the same as ID: 544506524794908672
ID: 544506659973136384 is the same as ID: 544506659973136384
ID: 544510385701662720 is the same as ID: 544510385701662720
ID: 544510279694823424 is the same as ID: 544510279694823424
ID: 544510576148238336 is the same as ID: 544510576148238336
ID: 544510767530524672 is the same as ID: 544510767530524672
ID: 544514420769296384 is the same as ID: 544514420769296384
ID: 544536420660961280 is the same as ID: 544536420660961280
ID: 544721484824842240 is the same as ID: 544721484824842240
ID: 544504183341064192 is the same as ID: 544504183341064192
ID: 544510638202966016 is the same as ID: 544510638202966016
ID: 544510896614412288 i

In [115]:
j = pd.concat(g for _, g in tweets.groupby("id") if len(g) > 1)
j.shape

(74, 27)

In [88]:
thing = pd.merge(annotations, tweets, on=['tweetid', 'id'], how='outer')

KeyError: 'tweetid'

In [123]:
annotations.merge(tweets, on='id').query('threadid == "in_reply_to_status_id"')

KeyError: 'id'

# Structural Features

In [ ]:
# Average tweet length

In [ ]:
# Thread lifetime (number of minutes bewtween first and last tweet)

In [ ]:
# Depth of the conversation thread

In [2]:
# Function to find retweets, url, media, mentions, hashtags, and tweets count
def makeCounts(tweets):
    retweetCount = 0
    urlCount = 0
    mediaCount = 0
    mentionCount = 0
    hashtagCount = 0
    tweetcount = 0
    
    for tweet in tweets:
        if("retweeted_status" in tweet):
            retweetCount += 1
            continue
        
        entityFlag = False
        
        entityList = tweet["entities"]
        if(len(entityList["user_mentions"]) > 0):
            mentionCount += 1
            entityFlag = True
        if(len(entityList['hashtags']) > 0):
            hashtagCount += 1
            entityFlag = True
        if(len(entityList["urls"]) > 0):
            urlCount += 1
            entityFlag = True
        if("media" in entityList):
            mediaCount += 1
            entityFlag = True
        if(not entityFlag):
            tweetCount += 1
        
    countMap = {
            "retweetCount" : retweetCount,
            "urlCount" : urlCount,
            "mediaCount" : mediaCount,
            "mentionCount" : menCtionCount,
            "tweetCount" : tweetCount
    }
    
    return countMap

SyntaxError: invalid syntax (<ipython-input-2-6796764e85ed>, line 10)

# User Features

In [ ]:
# Account age

In [ ]:
# Average follower count

In [ ]:
# Average friend count

In [ ]:
# Average status count

In [ ]:
# Frequency of verified authors

In [ ]:
# Whether the author of the first tweet in the thread is verified

In [ ]:
# Difference between when an account was created and the relevant tweet was authored

In [ ]:
# Network Density

# Content Features

In [ ]:
for eventKey in eventMap.keys():
    localG = nx.Graph()
    dl = eventMap[eventKey]
    
    densityMap = {}
    sentMap = {}
    objMap = {}
    
    sentimentList = []
    objectivityList = []
    
    for sliceTime i sorted(dl["tweets"].keys()):
        tweetList = dl["tweets"][sliceTime]
        if(len(tweetList) == 0):
            continue
        
        for tweet in tweetList:
            textblob = TextBlob(tweet["text"])
            sentiment = textBlob.sentiment
            
            sentimentList.append(sentiment.polarity)
            objectivityList.append(sentiment.subjectivity)
            
            

In [ ]:
# Polarity (The average positive or negative feelings expressed in a tweet)
plt.figure(figsize=(16,8))

trueData = []
falseData = []

for eventKey in eventMap.keys():
    localEvent = eventMap[eventKey]
    sentMap = localMap["polarity"]
    sortedTimes = sorted(sentMap.keys())
    

In [ ]:
# Subjectivity (a score of whether a tweet is objective or subjective)

In [ ]:
# Frequency and proportions of questions marks

In [ ]:
# Frequency and proportions of exclamation points

In [ ]:
# Frequency and proportions of first/second/third person pronouns

In [ ]:
# Frequency and proportions of smiling emoticons

# Temporal Features

In [ ]:
# Account age

In [ ]:
# Difference betwwen account age and tweet publication time

In [ ]:
# Author followers/friends/statuses

In [ ]:
# Number of tweets per minute